### Modelo de Recomendación

Este modelo tiene como objetivo evaluar la efectividad del método TfidfVectorizer para la recomendación de películas.

El propósito principal es demostrar su capacidad para identificar y sugerir productos similares, facilitando así la búsqueda y selección de opciones relevantes para los usuarios.

In [42]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import ast
import requests
import numpy as np

In [43]:
df = pd.read_csv(r'C:\Users\Carlo\Desktop\Proyectos\Data\TF-IDF Vectorizer\Dataset_peliculas2,0.csv')

In [44]:
df.columns

Index(['id', 'titulo', 'keywords', 'generos', 'rating', 'Idioma',
       'lenguaje_original', 'pais_de_creacion', 'overview', 'votos',
       'año_de_estreno', 'director', 'duracion'],
      dtype='object')

In [45]:
df.head()

,id,titulo,keywords,generos,rating,Idioma,lenguaje_original,pais_de_creacion,overview,votos,año_de_estreno,director,duracion
0,5,Four Rooms,"['hotel', ""new year's eve"", 'witch', 'bet', 's...",['Comedia'],5.830,en,en,United States of America,"Durante una Nochevieja, en un hotel de Los Áng...",2589,1995.0,Allison Anders,98.0
1,11,La guerra de las galaxias,"['empire', 'galaxy', 'rebellion', 'android', '...","['Aventura', 'Acción', 'Ciencia ficción']",8.203,es,en,United States of America,"La princesa Leia, líder del movimiento rebelde...",20248,1977.0,George Lucas,121.0
2,12,Buscando a Nemo,"['sydney, australia', 'parent child relationsh...","['Animación', 'Familia']",7.821,pt,en,United States of America,"Nemo, un pececillo, hijo único muy querido y p...",18921,2003.0,Andrew Stanton,100.0
3,13,Forrest Gump,"['vietnam war', 'vietnam veteran', 'mentally d...","['Comedia', 'Drama', 'Romance']",8.475,en,en,United States of America,Forrest Gump es un chico con deficiencias ment...,26915,1994.0,Robert Zemeckis,142.0
4,15,Ciudadano Kane,"['media tycoon', 'florida', 'art collector', '...","['Misterio', 'Drama']",8.009,es,en,United States of America,"Un importante financiero estadounidense, Charl...",5375,1941.0,Orson Welles,119.0


In [46]:
df['generos'] = df['generos'].apply(ast.literal_eval)
df['keywords'] = df['keywords'].apply(ast.literal_eval)

In [47]:
df['keywords'] = df['keywords'].apply(lambda x: ', '.join(x))
df['generos'] = df['generos'].apply(lambda x: ', '.join(x))

In [48]:
df['feature'] = (
    df['overview'].fillna('').astype(str) + 
    '' + df['director'].fillna('').astype(str) + 
    '' + df['generos'].fillna('').astype(str) +  
    '' + df['titulo'].fillna('').astype(str) 
)

#### Pruebas y Observaciones

Se llevaron a cabo pruebas incorporando la columna de keywords. Sin embargo, tras una inspección detallada, se observó que, en muchas películas, el contenido de esta columna carece de coherencia o relevancia, lo que podría influir negativamente en la precisión y efectividad de las recomendaciones generadas por el modelo.

#### Se ordenaron las películas según la cantidad de votos, lo que permitió seleccionar algunas de ellas para analizar y evaluar el comportamiento del modelo en términos de recomendación.

In [49]:
df.sort_values(by='votos', ascending=False).head(50)

,id,titulo,keywords,generos,rating,Idioma,lenguaje_original,pais_de_creacion,overview,votos,año_de_estreno,director,duracion,feature
35708,27205,Origen,"rescue, mission, dream, airplane, paris, franc...","Acción, Ciencia ficción, Aventura",8.368,de,en,United Kingdom,"Dom Cobb es un ladrón hábil, el mejor de todos...",36040,2010.0,Christopher Nolan,148.0,"Dom Cobb es un ladrón hábil, el mejor de todos..."
13706,157336,Interstellar,"rescue, future, spacecraft, race against time,...","Aventura, Drama, Ciencia ficción",8.436,de,en,United Kingdom,Narra las aventuras de un grupo de exploradore...,34780,2014.0,Christopher Nolan,169.0,Narra las aventuras de un grupo de exploradore...
77,155,El caballero oscuro,"joker, sadism, chaos, secret identity, crime f...","Drama, Acción, Crimen, Suspense",8.516,es,en,United Kingdom,Batman/Bruce Wayne regresa para continuar su g...,32247,2008.0,Christopher Nolan,152.0,Batman/Bruce Wayne regresa para continuar su g...
4459,19995,Avatar,"paraplegic, attachment to nature, culture clas...","Acción, Aventura, Fantasía, Ciencia ficción",7.582,en,en,United States of America,"Año 2154. Jake Sully, un exmarine en silla de ...",31083,2009.0,James Cameron,162.0,"Año 2154. Jake Sully, un exmarine en silla de ..."
5049,24428,Los Vengadores,"new york city, superhero, shield, based on com...","Ciencia ficción, Acción, Aventura",7.714,es,en,United States of America,Cuando un enemigo inesperado surge como una gr...,30286,2012.0,Joss Whedon,143.0,Cuando un enemigo inesperado surge como una gr...
17425,293660,Deadpool,"superhero, anti hero, mercenary, based on comi...","Acción, Aventura, Comedia",7.614,en,en,United States of America,Basado en el anti-héroe menos convencional de ...,30234,2016.0,Tim Miller,108.0,Basado en el anti-héroe menos convencional de ...
17640,299536,Vengadores: Infinity War,"sacrifice, magic, superhero, based on comic, s...","Aventura, Acción, Ciencia ficción",8.246,es,en,United States of America,El todopoderoso Thanos ha despertado con la pr...,29164,2018.0,Anthony Russo,149.0,El todopoderoso Thanos ha despertado con la pr...
262,550,El club de la lucha,"dual identity, rage and hate, based on novel o...",Drama,8.441,es,en,Germany,Un joven sin ilusiones lucha contra su insomni...,28843,1999.0,David Fincher,139.0,Un joven sin ilusiones lucha contra su insomni...
12464,118340,Guardianes de la galaxia,"spacecraft, based on comic, space, orphan, adv...","Acción, Ciencia ficción, Aventura",7.908,es,en,United States of America,El temerario aventurero Peter Quill es objeto ...,27640,2014.0,James Gunn,121.0,El temerario aventurero Peter Quill es objeto ...
335,680,Pulp Fiction,"drug dealer, boxer, massage, stolen money, bri...","Suspense, Crimen",8.488,en,en,United States of America,"Jules y Vincent, dos asesinos a sueldo con muy...",27409,1994.0,Quentin Tarantino,154.0,"Jules y Vincent, dos asesinos a sueldo con muy..."


In [50]:
tfidf = TfidfVectorizer()
onehot = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('feature', tfidf, 'feature'),
    ]
)

pipeline = make_pipeline(preprocessor, NearestNeighbors(n_neighbors=10))
pipeline.fit(df)

def recommend(movie_id):
    if movie_id not in df['id'].values:
        raise ValueError(f"El ID de la película {movie_id} no se encuentra en el DataFrame.")
    
    movie_index = df.index[df['id'] == movie_id].tolist()[0]
    distances, indices = pipeline.named_steps['nearestneighbors'].kneighbors(
        pipeline.named_steps['columntransformer'].transform(df.iloc[movie_index:movie_index+1])
    )

    recommended_movies = df.iloc[indices[0]].sort_values(by= 'votos', ascending=False)
    
    recommended_movies = recommended_movies[recommended_movies['id'] != movie_id]
    
    return recommended_movies


## Harry Potter y la cámara secreta

In [51]:
recommendations = recommend(672)

recommendations

,id,titulo,keywords,generos,rating,Idioma,lenguaje_original,pais_de_creacion,overview,votos,año_de_estreno,director,duracion,feature
328,671,Harry Potter y la piedra filosofal,"witch, school friend, friendship, london, engl...","Aventura, Fantasía",7.913,es,en,United Kingdom,Harry Potter es un huérfano que vive con sus d...,26836,2001.0,Chris Columbus,152.0,Harry Potter es un huérfano que vive con sus d...
330,673,Harry Potter y el prisionero de Azkaban,"witch, school friend, friendship, flying, magi...","Aventura, Fantasía",8.018,es,en,United Kingdom,Harry está deseando que termine el verano para...,21123,2004.0,Alfonso Cuarón,141.0,Harry está deseando que termine el verano para...
331,674,Harry Potter y el cáliz de fuego,"witch, dying and death, based on novel or book...","Aventura, Fantasía",7.816,es,en,United Kingdom,"En el cuarto año en Hogwarts, Harry se enfrent...",20278,2005.0,Mike Newell,157.0,"En el cuarto año en Hogwarts, Harry se enfrent..."
2797,12445,Harry Potter y las Reliquias de la Muerte - Pa...,"witch, dying and death, saving the world, self...","Fantasía, Aventura",8.098,es,en,United Kingdom,La segunda parte de la batalla final entre las...,20176,2011.0,David Yates,130.0,La segunda parte de la batalla final entre las...
332,675,Harry Potter y la Orden del Fénix,"witch, dying and death, court, magic, prophecy...","Aventura, Fantasía",7.682,es,en,United Kingdom,Harry Potter regresa por quinto año a Hogwarts...,19086,2007.0,David Yates,138.0,Harry Potter regresa por quinto año a Hogwarts...
363,767,Harry Potter y el misterio del príncipe,"witch, dying and death, london, england, magic...","Aventura, Fantasía",7.700,es,en,United Kingdom,En medio de los desastres que azotan a Inglate...,19063,2009.0,David Yates,153.0,En medio de los desastres que azotan a Inglate...
2796,12444,Harry Potter y las Reliquias de la Muerte - Pa...,"witch, friendship, london, england, corruption...","Aventura, Fantasía",7.755,es,en,United Kingdom,Una tarea casi imposible cae sobre los hombros...,18703,2010.0,David Yates,146.0,Una tarea casi imposible cae sobre los hombros...
3199,13967,Miss Potter,"mountain lake, loss of loved one, biography, a...","Drama, Familia, Romance",6.601,en,en,United Kingdom,Beatrix Potter fue una mujer inteligente y car...,477,2006.0,Chris Noonan,92.0,Beatrix Potter fue una mujer inteligente y car...
24520,506600,Una foto antes de morir,,"Suspense, Terror",5.241,es,ru,Russia,Un grupo de amigos decide pasar un fin de sema...,54,2018.0,Anton Zenkovich,90.0,Un grupo de amigos decide pasar un fin de sema...


## La Guerra de las Galaxias de 1977

In [52]:
recommendations = recommend(11)

recommendations

,id,titulo,keywords,generos,rating,Idioma,lenguaje_original,pais_de_creacion,overview,votos,año_de_estreno,director,duracion,feature
13327,140607,Star Wars: El despertar de la fuerza,"android, spacecraft, space opera","Aventura, Acción, Ciencia ficción",7.281,es,en,United States of America,Treinta años después de la victoria de la Alia...,19069,2015.0,J.J. Abrams,136.0,Treinta años después de la victoria de la Alia...
673,1891,El imperio contraataca,"rebellion, android, spacecraft, asteroid, rebe...","Aventura, Acción, Ciencia ficción",8.390,es,en,United States of America,Tras un ataque sorpresa de las tropas imperial...,16634,1980.0,Irvin Kershner,124.0,Tras un ataque sorpresa de las tropas imperial...
674,1892,El retorno del Jedi,"spacecraft, sibling relationship, rebel, emper...","Aventura, Acción, Ciencia ficción",7.895,es,en,United States of America,Luke Skywalker y la princesa Leia deben viajar...,15390,1983.0,Richard Marquand,132.0,Luke Skywalker y la princesa Leia deben viajar...
675,1893,La guerra de las galaxias. Episodio I: La amen...,"galaxy, senate, taskmaster, prophecy, queen, a...","Aventura, Acción, Ciencia ficción",6.550,es,en,United States of America,La Federación de Comercio ha bloqueado el pequ...,14316,1999.0,George Lucas,136.0,La Federación de Comercio ha bloqueado el pequ...
677,1895,La guerra de las galaxias. Episodio III: La ve...,"showdown, lava, fight, politics, volcano, chos...","Aventura, Acción, Ciencia ficción",7.427,es,en,United States of America,"Último capítulo de la saga de Star Wars, en el...",13437,2005.0,George Lucas,140.0,"Último capítulo de la saga de Star Wars, en el..."
676,1894,La guerra de las galaxias. Episodio II: El ata...,"army, laser gun, senate, investigation, cult f...","Aventura, Acción, Ciencia ficción",6.562,es,en,United States of America,Corren tenebrosos tiempos para la República. É...,12968,2002.0,George Lucas,142.0,Corren tenebrosos tiempos para la República. É...
29507,724475,Ben 10 contra el Universo: La película,"extraterrestrial technology, alien, based on c...","Animación, Acción, Aventura, Ciencia ficción",7.500,ca,en,United States of America,"La historia de esta nueva cinta animada, narra...",119,2020.0,Henrique Jardim,70.0,"La historia de esta nueva cinta animada, narra..."
19639,362610,I Am Your Father,"movie business, interview, makeup, space opera...",Documental,6.600,en,es,Spain,"Tributo en honor a David Prowse, el hombre baj...",51,2015.0,Toni Bestard,83.0,"Tributo en honor a David Prowse, el hombre baj..."
26311,570871,La Ultima galaxia,"male friendship, space travel, alien planet, a...","Drama, Ciencia ficción",5.421,es,de,Germany,"Adem y Novak, dos jóvenes muy diferentes, esca...",20,2021.0,Marcel Barion,113.0,"Adem y Novak, dos jóvenes muy diferentes, esca..."


## El señor de los anillos: Las dos torres

In [53]:
recommendations = recommend(121)

recommendations

,id,titulo,keywords,generos,rating,Idioma,lenguaje_original,pais_de_creacion,overview,votos,año_de_estreno,director,duracion,feature
56,120,El señor de los anillos: La comunidad del anillo,"based on novel or book, orcs, elves, dwarf, ma...","Aventura, Fantasía, Acción",8.413,es,en,New Zealand,"En la Tierra Media, el Señor Oscuro Saurón cre...",24606,2001.0,Peter Jackson,179.0,"En la Tierra Media, el Señor Oscuro Saurón cre..."
58,122,El señor de los anillos: El retorno del rey,"army, based on novel or book, orcs, elves, dwa...","Aventura, Fantasía, Acción",8.481,es,en,New Zealand,"Las fuerzas de Saruman han sido destruidas, y ...",23661,2003.0,Peter Jackson,201.0,"Las fuerzas de Saruman han sido destruidas, y ..."
7921,49051,El hobbit: Un viaje inesperado,"based on novel or book, orcs, elves, dwarf, ma...","Aventura, Fantasía, Acción",7.354,es,en,New Zealand,"Precuela de la trilogía ""El Señor de los Anill...",18067,2012.0,Peter Jackson,169.0,"Precuela de la trilogía ""El Señor de los Anill..."
3444,14787,Tom y Jerry: el anillo mágico,magic ring,"Animación, Comedia, Familia, Fantasía",6.400,es,en,United States of America,Habiéndole encargado la custodia del anillo má...,243,2002.0,James T. Walker,62.0,Habiéndole encargado la custodia del anillo má...
3194,13957,Cerrando el círculo,"airplane, world war ii, bomber, love, soldier,...","Drama, Romance",6.100,es,en,Canada,En 1943 un soldado de artillería moribundo tra...,94,2007.0,Richard Attenborough,118.0,En 1943 un soldado de artillería moribundo tra...
23147,462677,Doraemon y la gran aventura en la Antártida,"time travel, antarctica","Animación, Aventura, Fantasía, Familia",6.494,es,ja,Japan,"Incapaz de soportar el calor del verano, Dorae...",88,2017.0,Atsushi Takahashi,101.0,"Incapaz de soportar el calor del verano, Dorae..."
29059,684476,¡Devuelve el anillo!,,"Comedia, Romance, Drama",7.600,es,he,Israel,Los novios llegan a su lujosa suite de luna de...,11,2020.0,Talya Lavie,90.0,Los novios llegan a su lujosa suite de luna de...
31508,850367,Tropical-Rouge! Pretty Cure: ¡La Princesa de l...,"superhero, snow, precure, supervillain, based ...","Animación, Fantasía, Acción, Comedia, Aventura...",7.100,es,ja,Japan,"Un día, las Pretty Cure de Tropical-Rouge reci...",5,2021.0,Junji Shimizu,70.0,"Un día, las Pretty Cure de Tropical-Rouge reci..."
32313,919345,Ring of Desire,"eroticism, sensuality, ring, lesbian kiss","Suspense, Terror",3.000,en,en,United States of America,Un antiguo anillo que otorga poder sobre quien...,2,2021.0,Ronnie Nanos,85.0,Un antiguo anillo que otorga poder sobre quien...


## Malditos Bastardos

In [54]:
recommendations = recommend(16869)

recommendations

,id,titulo,keywords,generos,rating,Idioma,lenguaje_original,pais_de_creacion,overview,votos,año_de_estreno,director,duracion,feature
1590,9075,El libro negro,"in love with enemy, world war ii, prosecution,...","Drama, Suspense, Bélica",7.470,es,nl,Belgium,Segunda Guerra Mundial (1939-1945). Tras la ej...,1111,2006.0,Paul Verhoeven,145.0,Segunda Guerra Mundial (1939-1945). Tras la ej...
16759,271674,Suite francesa,"based on novel or book, world war ii, prisoner...","Drama, Romance, Bélica",7.284,es,en,Belgium,"Año 1940, durante la ocupación alemana del ejé...",1084,2015.0,Saul Dibb,103.0,"Año 1940, durante la ocupación alemana del ejé..."
2240,10592,La guerra de Hart,"escape, black people, world war ii, prisoner o...","Drama, Bélica",6.400,es,en,United States of America,"Durante la Segunda Guerra Mundial, el teniente...",902,2002.0,Gregory Hoblit,125.0,"Durante la Segunda Guerra Mundial, el teniente..."
30756,799583,El Ministerio de la Guerra Sucia,"army, operation, based on novel or book, spy, ...","Acción, Bélica, Drama",7.277,es,en,United Kingdom,El ejército británico recluta a un pequeño gru...,452,2024.0,Guy Ritchie,120.0,El ejército británico recluta a un pequeño gru...
1353,8056,El tren de la vida,"nazi, concentration camp, holocaust (shoah), v...","Drama, Comedia, Bélica",7.346,es,fr,Belgium,"Durante la Segunda Guerra Mundial (1939-1945),...",267,1998.0,Radu Mihăileanu,103.0,"Durante la Segunda Guerra Mundial (1939-1945),..."
16650,268340,Las madres del Tercer Reich,"factory, suicide, world war ii, race politics,...","Drama, Bélica, Película de TV",7.333,es,fr,France,"Basado en el libro de Nina Barbier, Les Malgré...",39,2012.0,Denis Malleval,92.0,"Basado en el libro de Nina Barbier, Les Malgré..."
35308,848730,Los bastardos,,"Acción, Suspense, Crimen, Drama",6.500,es,es,Argentina,Una epopeya sobre una de las épocas más convul...,21,2023.0,Pablo Yotich,114.0,Una epopeya sobre una de las épocas más convul...
22118,433829,Star Raiders: The Adventures of Saber Raine,"mercenary, alien, outcast, creature, robot, ro...","Aventura, Ciencia ficción",3.658,en,en,United States of America,El aventurero estelar Saber Raine es contratad...,19,2017.0,Mark Steven Grove,83.0,El aventurero estelar Saber Raine es contratad...
20366,384316,La paz salvaje,"concentration camp prisoner, czechoslovakia, e...","Documental, Historia, Película de TV",7.400,es,en,United Kingdom,"En 1945, después del final de la Segunda Guerr...",9,2015.0,Peter Molloy,60.0,"En 1945, después del final de la Segunda Guerr..."


## Capitán América: Civil War

In [55]:
recommendations = recommend(271110)

recommendations

,id,titulo,keywords,generos,rating,Idioma,lenguaje_original,pais_de_creacion,overview,votos,año_de_estreno,director,duracion,feature
5049,24428,Los Vengadores,"new york city, superhero, shield, based on com...","Ciencia ficción, Acción, Aventura",7.714,es,en,United States of America,Cuando un enemigo inesperado surge como una gr...,30286,2012.0,Joss Whedon,143.0,Cuando un enemigo inesperado surge como una gr...
17640,299536,Vengadores: Infinity War,"sacrifice, magic, superhero, based on comic, s...","Aventura, Acción, Ciencia ficción",8.246,es,en,United States of America,El todopoderoso Thanos ha despertado con la pr...,29164,2018.0,Anthony Russo,149.0,El todopoderoso Thanos ha despertado con la pr...
17639,299534,Vengadores: Endgame,"superhero, time travel, space travel, time mac...","Aventura, Ciencia ficción, Acción",8.253,gl,en,United States of America,Después de los eventos devastadores de 'Vengad...,25125,2019.0,Anthony Russo,181.0,Después de los eventos devastadores de 'Vengad...
642,1771,Capitán América: El primer vengador,"new york city, nazi, superhero, world war ii, ...","Acción, Aventura, Ciencia ficción",6.999,es,en,United States of America,"Nacido durante la Gran Depresión, Steve Rogers...",21073,2011.0,Joe Johnston,124.0,"Nacido durante la Gran Depresión, Steve Rogers..."
11676,100402,Capitán América: El soldado de invierno,"washington dc, usa, superhero, shield, based o...","Acción, Aventura, Ciencia ficción",7.664,es,en,United States of America,"Capitán América, Viuda Negra y un nuevo aliado...",18442,2014.0,Anthony Russo,136.0,"Capitán América, Viuda Negra y un nuevo aliado..."
3397,14611,Ultimate Avengers 2: Rise of the Panther,"mask, africa, alien life-form, based on comic,...","Aventura, Animación, Acción, Ciencia ficción",6.800,en,en,United States of America,"Secuela de ""Ultimate Avengers"" lanzada directa...",284,2006.0,Dick Sebast,73.0,"Secuela de ""Ultimate Avengers"" lanzada directa..."
3398,14613,Nuevos Vengadores: Heroes del mañana,"based on comic, superhero team","Animación, Familia, Acción, Aventura, Ciencia ...",6.945,es,en,United States of America,Cada vez que las fuerzas del mal amenazaron a ...,260,2008.0,Jay Oliva,78.0,Cada vez que las fuerzas del mal amenazaron a ...
25474,538153,Los Nuevos Vengadores,,"Comedia, Fantasía, Acción",5.600,es,en,United States of America,Mientras intenta seguir siendo un buen esposo ...,24,2018.0,Jarret Tarnol,88.0,Mientras intenta seguir siendo un buen esposo ...
32221,912223,Los Vengadores Chiflados,"los vengadores, los vengadores chiflados, jepc...","Comedia, Acción, Aventura, Ciencia ficción",10.000,es,es,Mexico,Película de juguetes que parodia a los vengado...,1,2014.0,Jesús Zavala,65.0,Película de juguetes que parodia a los vengado...


### Primeras Conclusiones

El modelo muestra un nivel aceptable de funcionamiento, realizando recomendaciones con cierto grado de coherencia. Aunque es evidente que existen áreas de mejora, los resultados iniciales indican un desempeño prometedor.

### Procedemos a evaluar que parecido guarda este modelo y el oficial de TMDB


In [56]:
idx_pos = df.sort_values(by='votos', ascending=False).head(50)

In [57]:
idx_pos = idx_pos['id'].values

In [58]:
idx_pos.tolist()

[27205,
 157336,
 155,
 19995,
 24428,
 293660,
 299536,
 550,
 118340,
 680,
 13,
 671,
 278,
 1726,
 68718,
 603,
 299534,
 120,
 122,
 11324,
 106646,
 99861,
 271110,
 49026,
 76341,
 284054,
 16869,
 68721,
 284052,
 70160,
 672,
 315635,
 121,
 150540,
 283995,
 673,
 1771,
 297761,
 10195,
 10138,
 272,
 284053,
 674,
 11,
 22,
 12445,
 238,
 135397,
 14160,
 297762]

In [59]:
df_recomn = pd.DataFrame(columns=['id_inicial', 'id_recomendado'])

for movie_idx in idx_pos:
    try:
        if movie_idx not in df['id'].values:
            print(f"El ID de la película {movie_idx} no se encuentra en el DataFrame. Se omite este ID.")
            continue

        recommendations = recommend(movie_idx)
        
        recommended_ids = recommendations['id'].tolist() 
        
        df_tmp = pd.DataFrame({'id_inicial': [movie_idx], 'id_recomendado': [recommended_ids]})
        
        df_recomn = pd.concat([df_recomn, df_tmp], ignore_index=True)
    
    except ValueError as e:
        print(e)
        continue

df_recomn    

,id_inicial,id_recomendado
0,27205,"[577922, 9469, 470114, 192134, 1546, 305937, 5..."
1,157336,"[70981, 540247, 449575, 429101, 493522, 451349..."
2,155,"[49026, 272, 40662, 142061, 16234, 251519, 736..."
3,19995,"[76600, 10196, 681887, 728526, 183392, 243451,..."
4,24428,"[299534, 271110, 141052, 246655, 14613, 27460,..."
5,293660,"[383498, 8457, 567604, 9087, 31662, 565470, 77..."
6,299536,"[299534, 99861, 271110, 284054, 283995, 324857..."
7,550,"[463257, 1913, 2925, 800787, 12400, 682377, 44..."
8,118340,"[299536, 283995, 447365, 698, 168, 517096, 604..."
9,680,"[510388, 738231, 767896, 369054, 170178, 88118..."


In [60]:
API_KEY = '6e654f8882a28bd2b44888b6acef8373'
BASE_URL = 'https://api.themoviedb.org/3'

def get_recommendations(movie_id, api_key=API_KEY):
    url = f'{BASE_URL}/movie/{movie_id}/recommendations'
    params = {
        'api_key': api_key,
        'language': 'es-ES',
        'page': 1 
    }
    response = requests.get(url, params=params)
    data = response.json()
    recommendations = data.get('results', [])[:10] 
    return recommendations

def get_movie_recommendations(movie_ids, api_key=API_KEY):
    all_recommendations = {}
    for movie_id in movie_ids:
        try:
            recommendations = get_recommendations(movie_id, api_key)
            recommended_ids = [rec['id'] for rec in recommendations]
            all_recommendations[movie_id] = recommended_ids
        except Exception as e:
            print(f"Error obteniendo recomendaciones para la película {movie_id}: {e}")
    return all_recommendations

def recommendations_to_dataframe(recommendations):
    df_list = []
    for movie_id, recs in recommendations.items():
        df_list.append({
            'id_inicial': movie_id,
            'recomendaciones': recs 
        })
    
    df_recomn = pd.DataFrame(df_list)
    return df_recomn

recommendations = get_movie_recommendations(idx_pos)

df_recomn_2_tmdb = recommendations_to_dataframe(recommendations)

df_recomn_2_tmdb

,id_inicial,recomendaciones
0,27205,"[11324, 155, 157336, 70160, 603, 68721, 19995,..."
1,157336,"[27205, 286217, 118340, 78, 210577, 206487, 15..."
2,155,"[49026, 272, 680, 122, 550, 27205, 1726, 13, 1..."
3,19995,"[24428, 68721, 27205, 76600, 49026, 70160, 377..."
4,24428,"[99861, 68721, 81005, 37724, 19995, 1771, 7016..."
5,293660,"[383498, 271110, 284052, 209112, 140607, 11834..."
6,299536,"[299534, 363088, 284054, 383498, 99861, 24428,..."
7,550,"[680, 807, 13, 59967, 603, 155, 510, 73, 2649,..."
8,118340,"[283995, 99861, 100402, 127585, 240832, 119450..."
9,680,"[13, 500, 16869, 550, 24, 68718, 122, 155, 278..."


### Teniendo los dos dataframe, se puede calcular si ambos tienen la capacidad de recomendar peliculas similares

In [61]:
import pandas as pd

# Función para contar coincidencias entre dos listas
def count_common_elements(list1, list2):
    """Cuenta los elementos comunes entre dos listas."""
    if isinstance(list1, list) and isinstance(list2, list):
        # Desanidar las listas si es necesario
        list1 = [item for sublist in list1 for item in (sublist if isinstance(sublist, list) else [sublist])]
        list2 = [item for sublist in list2 for item in (sublist if isinstance(sublist, list) else [sublist])]
        set1 = set(list1)
        set2 = set(list2)
        return len(set1 & set2)
    return 0

# Agrupar `df_recomn` por `id_inicial` y combinar las recomendaciones en una lista
df_recomn_grouped = df_recomn.groupby('id_inicial')['id_recomendado'].apply(lambda x: [item for sublist in x for item in (sublist if isinstance(sublist, list) else [sublist])]).reset_index()

# Convertir `df_recomn_2_tmdb` para asegurar que las recomendaciones estén en formato lista
df_recomn_2_tmdb['recomendaciones'] = df_recomn_2_tmdb['recomendaciones'].apply(lambda x: [item for sublist in x for item in (sublist if isinstance(sublist, list) else [sublist])])

# Hacer merge de los dos DataFrames en base a `id_inicial`
merged_df = pd.merge(df_recomn_grouped, df_recomn_2_tmdb, on='id_inicial')

# Aplicar la función para contar coincidencias
merged_df['coincidencias'] = merged_df.apply(
    lambda row: count_common_elements(row['id_recomendado'], row['recomendaciones']), axis=1
)

# Mostrar el DataFrame con las coincidencias
merged_df



,id_inicial,id_recomendado,recomendaciones,coincidencias
0,11,"[140607, 1891, 1892, 1893, 1895, 1894, 724475,...","[1891, 1892, 13475, 140607, 1893, 181808, 122,...",5
1,13,"[239877, 342474, 244534, 381040, 49928, 428849...","[497, 550, 122, 857, 769, 597, 637, 429, 197, ...",0
2,22,"[58, 285, 172385, 323272, 399747, 531836, 5016...","[58, 285, 1865, 166426, 8373, 4133, 12, 162, 1...",2
3,120,"[122, 121, 49051, 14787, 13957, 462677, 684476...","[122, 121, 27205, 10195, 603, 1891, 68734, 172...",3
4,121,"[120, 122, 49051, 14787, 13957, 462677, 684476...","[122, 120, 672, 49051, 807, 1891, 14869, 272, ...",3
5,122,"[120, 121, 49051, 1892, 122917, 23452, 14763, ...","[121, 120, 49051, 562, 13, 429, 603, 82690, 36...",3
6,155,"[49026, 272, 40662, 142061, 16234, 251519, 736...","[49026, 272, 680, 122, 550, 27205, 1726, 13, 1...",2
7,238,"[240, 242, 252838, 11975, 598065, 61899, 11787...","[240, 424, 278, 242, 680, 111, 769, 429, 389, ...",2
8,272,"[155, 49026, 414, 14919, 251519, 69735, 177271...","[155, 49026, 414, 209112, 364, 268, 1726, 857,...",3
9,278,"[6071, 14851, 471859, 530956, 38124, 58102, 49...","[238, 424, 240, 680, 155, 497, 13, 389, 274, 550]",0


In [62]:
merged_df['coincidencias'].sum()

np.int64(92)

### Comparativa con el Modelo de TMDB

En la comparación entre el modelo desarrollado y el motor de recomendación de TMDB, se encontraron 92 coincidencias entre las 500 recomendaciones generadas por ambos modelos. Sin embargo, tras una revisión preliminar del sistema de TMDB, se identificó un margen significativo de error en sus recomendaciones. A partir de la tercera sugerencia, el motor de TMDB tiende a desviarse notablemente en términos de género y sinopsis, lo que genera recomendaciones poco relevantes.

Este análisis pone de manifiesto que el motor de TMDB no puede considerarse una base confiable, dado que un porcentaje considerable de sus recomendaciones carece de sentido. Es importante señalar que esta evaluación se basó en búsquedas manuales, donde únicamente las primeras cinco recomendaciones mostraron un nivel aceptable de precisión.

### Comparativa con el Modelo de TMDB

En la comparación entre el modelo desarrollado y el motor de recomendación de TMDB, se encontraron 92 coincidencias entre las 500 recomendaciones generadas por ambos modelos. Sin embargo, tras una revisión preliminar del sistema de TMDB, se identificó un margen significativo de error en sus recomendaciones. A partir de la tercera sugerencia, el motor de TMDB tiende a desviarse notablemente en términos de género y sinopsis, lo que genera recomendaciones poco relevantes.

Este análisis pone de manifiesto que el motor de TMDB no puede considerarse una base confiable, dado que un porcentaje considerable de sus recomendaciones carece de sentido. Es importante señalar que esta evaluación se basó en búsquedas manuales, donde únicamente las primeras cinco recomendaciones mostraron un nivel aceptable de precisión.